<a href="https://colab.research.google.com/github/vpleshko-lab/camp-tasks/blob/main/lesson_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np



In [3]:
path = '/content/creditcard.csv'
df = pd.read_csv(path)
df.drop(columns=['Time'], inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 30 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V1      284807 non-null  float64
 1   V2      284807 non-null  float64
 2   V3      284807 non-null  float64
 3   V4      284807 non-null  float64
 4   V5      284807 non-null  float64
 5   V6      284807 non-null  float64
 6   V7      284807 non-null  float64
 7   V8      284807 non-null  float64
 8   V9      284807 non-null  float64
 9   V10     284807 non-null  float64
 10  V11     284807 non-null  float64
 11  V12     284807 non-null  float64
 12  V13     284807 non-null  float64
 13  V14     284807 non-null  float64
 14  V15     284807 non-null  float64
 15  V16     284807 non-null  float64
 16  V17     284807 non-null  float64
 17  V18     284807 non-null  float64
 18  V19     284807 non-null  float64
 19  V20     284807 non-null  float64
 20  V21     284807 non-null  float64
 21  V22     28

In [4]:
df.drop_duplicates(inplace=True)
X = df.drop(columns=['Class'])
y_true = df['Class']
print(df['Class'].value_counts())
X.head()

Class
0    275190
1       473
Name: count, dtype: int64


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [10]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_recall_curve, auc, confusion_matrix, make_scorer, f1_score

In [16]:
X_scaled = StandardScaler().fit_transform(X)
X_pca = PCA(n_components=2).fit_transform(X_scaled)

X_train, X_test, y_train, y_test = train_test_split(X_pca, y_true,
                                                    test_size=0.25,
                                                    stratify=y_true)
for name, arr in zip(['X_train','X_test','y_train','y_test'],[X_train, X_test, y_train, y_test]):
  print(f'{name}: {arr.shape}')

X_train: (206747, 2)
X_test: (68916, 2)
y_train: (206747,)
y_test: (68916,)


In [22]:
def run_ee():

  pipeline_ee = Pipeline([
      ('scaler', StandardScaler()),
      ('pca', PCA(n_components=0.95)),
      ('model', EllipticEnvelope(contamination=0.0017)) ])

  pipeline_ee.fit(X_train)

  labels = pipeline_ee.predict(X_test)
  scores = pipeline_ee.decision_function(X_test)

  y_pred = (labels == -1).astype(int)

  # evaluate
  print(classification_report(y_test, y_pred))
  roc_auc = roc_auc_score(y_test,-scores)
  print(f'ROC AUC: {roc_auc:.2f}')
  precision, recall, _ = precision_recall_curve(y_test, -scores)
  pr_auc = auc(recall, precision)
  print(f'PrecisionRecall AUC: {pr_auc:.2f}')

  cm = confusion_matrix(y_test, y_pred)
  print(cm)
run_ee()

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     68798
           1       0.00      0.00      0.00       118

    accuracy                           1.00     68916
   macro avg       0.50      0.50      0.50     68916
weighted avg       1.00      1.00      1.00     68916

ROC AUC: 0.70
PrecisionRecall AUC: 0.00
[[68699    99]
 [  118     0]]


In [24]:
def run_IF():

  pipeline = Pipeline([
      ('scaler', StandardScaler()),
      #('pca', PCA(n_components=0.95)),
      ('model', IsolationForest(contamination=0.0017, n_estimators=100)) ])

  pipeline.fit(X_train)

  y_pred = pipeline.predict(X_test)
  y_pred = [0 if x == 1 else 1 for x in y_pred]
  print(classification_report(y_test, y_pred, target_names=['non-fraud', 'fraud']))

  cm = confusion_matrix(y_test, y_pred)
  print(cm)

run_IF()

              precision    recall  f1-score   support

   non-fraud       1.00      1.00      1.00     68798
       fraud       0.00      0.00      0.00       118

    accuracy                           1.00     68916
   macro avg       0.50      0.50      0.50     68916
weighted avg       1.00      1.00      1.00     68916

[[68702    96]
 [  118     0]]


In [9]:
def run_LOF(X):
  model = LocalOutlierFactor(n_neighbors=25, contamination=0.0017)
  model.fit(X)

  y_pred = model.fit_predict(X)
  y_pred = [0 if x == 1 else 1 for x in y_pred]
  print(classification_report(y_true, y_pred, target_names=['non-fraud', 'fraud']))

run_LOF(X_pca)

              precision    recall  f1-score   support

   non-fraud       1.00      1.00      1.00    275190
       fraud       0.02      0.02      0.02       473

    accuracy                           1.00    275663
   macro avg       0.51      0.51      0.51    275663
weighted avg       1.00      1.00      1.00    275663

